In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F  
import urllib.request
import os
import numpy as np
import tarfile
import shutil
from PIL import Image
import glob
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
import matplotlib.pyplot as plt
import torchvision.models as models
%matplotlib inline

In [ ]:
from tqdm import tqdm

from google.colab import drive

if torch.cuda.is_available():
  device=torch.device('cuda:0')
  print('Cuda')
else:
  device=torch.device('cpu')
  print('cpu')


Cuda


In [ ]:

url='http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar'
path='VOC'

In [ ]:
def get_archive(path,url):
  try:
    os.mkdir(path)
  except:
    path=path
  
  filename='devkit'
  urllib.request.urlretrieve(url,f"{path}/{filename}.tar")
 
get_archive(path,url)


In [ ]:
def extract(path):
  tar_file=tarfile.open(f"{path}/devkit.tar")
  tar_file.extractall('./')
  tar_file.close()
  shutil.rmtree(path)

extract(path)

In [ ]:
VOC_COLORMAP = [
    [0, 0, 0],
    [128, 0, 0],
    [0, 128, 0],
    [128, 128, 0],
    [0, 0, 128],
    [128, 0, 128],
    [0, 128, 128],
    [128, 128, 128],
    [64, 0, 0],
    [192, 0, 0],
    [64, 128, 0],
    [192, 128, 0],
    [64, 0, 128],
    [192, 0, 128],
    [64, 128, 128],
    [192, 128, 128],
    [0, 64, 0],
    [128, 64, 0],
    [0, 192, 0],
    [128, 192, 0],
    [0, 64, 128],
]


In [ ]:
class VocDataset(Dataset):
  def __init__(self,dir,color_map):
    self.root=os.path.join(dir,'VOCdevkit/VOC2012')
    self.target_dir=os.path.join(self.root,'SegmentationClass')
    self.images_dir=os.path.join(self.root,'JPEGImages')
    file_list=os.path.join(self.root,'ImageSets/Segmentation/trainval.txt')
    self.files = [line.rstrip() for line in tuple(open(file_list, "r"))]
    self.color_map=color_map
    # self.pallete=self.get_collors()


  def convert_to_segmentation_mask(self,mask):
  # This function converts color channels of semgentation masks to number of classes (21 in this case)
  # Semantic Segmentation requires a segmentation mask to be a NumPy array with the shape [height, width, num_classes].
  # Each channel in this mask should encode values for a single class. Pixel in a mask channel should have
  # a value of 1.0 if the pixel of the image belongs to this class and 0.0 otherwise.
    height, width = mask.shape[:2]
    segmentation_mask = np.zeros((height, width, len(self.color_map)), dtype=np.float32)
    for label_index, label in enumerate(self.color_map):
          segmentation_mask[:, :, label_index] = np.all(mask == label, axis=-1).astype(float)
    return segmentation_mask

  def __getitem__(self,index):
    image_id=self.files[index]
    image_path=os.path.join(self.images_dir,f"{image_id}.jpg")
    label_path=os.path.join(self.target_dir,f"{image_id}.png")
    image=cv.imread(image_path)
    image=cv.cvtColor(image,cv.COLOR_BGR2RGB)
    image=cv.resize(image,(256,256))
    image=torch.tensor(image).float()
    label=cv.imread(label_path)
    label=cv.cvtColor(label,cv.COLOR_BGR2RGB)
    label=cv.resize(label,(256,256))
    label = self.convert_to_segmentation_mask(label)
    label=torch.tensor(label).float()
    
    return image,label


  
  def __len__(self):
    return len(self.files)


In [ ]:
data=VocDataset('/content',VOC_COLORMAP)
# plt.imshow(data.__getitem__(226)[0]/255)

In [ ]:
train_set,val_set=torch.utils.data.random_split(data,[int(len(data)*0.8),round(len(data)*0.2)])
train_loader=DataLoader(train_set,batch_size=10,shuffle=True)
val_loader=DataLoader(val_set,batch_size=10,shuffle=False)

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch


  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-p53yi4xr
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-p53yi4xr
     |████████████████████████████████| 58 kB 3.9 MB/s 
     |████████████████████████████████| 376 kB 15.3 MB/s 
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.2.0-py3-none-any.whl size=88681 sha256=8db381cb36a2f77edf859a85a46108584d18cfa5f3c7d9512bf096cb3cd2ac05
  Stored in directory: /tmp/pip-ephem-wheel-cache-6k7p5fyt/wheels/fa/c5/a8/1e8af6cb04a0974db8a4a156ebd2fdd1d99ad2558d3fce49d4
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=85abba273273760ec57d03fb4843b4b2b721299170a492fbe22bd57cfefa44d7
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-n

In [ ]:
import segmentation_models_pytorch as smp
model = smp.Unet(encoder_name='resnet18',classes=21,activation='softmax')
model=model.to(device)
criterion = smp.utils.losses.DiceLoss(eps=1.)
metrics = smp.utils.metrics.IoU(eps=1.)

optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)

In [ ]:
def train(model,optim,loss_f,epochs):
  
  for epoch in tqdm(range(epochs)):
    for (X_train,y_train) in train_loader:
      X_train,y_train=X_train.to(device),y_train.to(device,dtype=torch.int64)
      X_train = X_train.permute(0, 3, 1, 2)
      y_train = y_train.permute(0, 3, 1, 2)
      y_pred=model(X_train)
      loss=loss_f(y_pred,y_train)

      optim.zero_grad()
      loss.backward()
      optim.step()
    ious=[]
    val_losses=[]
    with torch.no_grad():
      for b,(X_test,y_test) in enumerate(val_loader):
        X_test,y_test=X_test.to(device),y_test.to(device)
        X_test = X_test.permute(0, 3, 1, 2)
        y_test = y_test.permute(0, 3, 1, 2)
        y_val=model(X_test)
        val_loss=loss_f(y_val,y_test)
        val_losses.append(val_loss)
        iou_=metrics(y_val,y_test)
        ious.append(iou_)
      ious=torch.tensor(ious)
      val_losses=torch.tensor(val_losses)

    print(f"epoch : {epoch:2} train_loss: {loss:10.4} , val_loss : {val_losses.mean():10.4} val_iou: {ious.mean()}")


train(model,optimizer,criterion,5)



  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/modules.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


 20%|██        | 1/5 [03:44<14:56, 224.15s/it]

epoch :  0 train_loss:     0.6489 , val_loss :     0.5817 val_iou: 0.44954782724380493




 40%|████      | 2/5 [07:28<11:12, 224.30s/it]

epoch :  1 train_loss:      0.224 , val_loss :     0.3055 val_iou: 0.6289107203483582




 60%|██████    | 3/5 [11:13<07:28, 224.41s/it]

epoch :  2 train_loss:     0.2568 , val_loss :     0.2641 val_iou: 0.6320332288742065




 80%|████████  | 4/5 [14:56<03:43, 223.86s/it]

epoch :  3 train_loss:     0.1777 , val_loss :     0.2497 val_iou: 0.6409472227096558




100%|██████████| 5/5 [18:39<00:00, 223.94s/it]

epoch :  4 train_loss:     0.1746 , val_loss :      0.247 val_iou: 0.6415237188339233
